# Verificacion de Estimaciones para la Campaña 2023.

In [361]:
import pandas as pd
import numpy as np
import math

# Estimacion de Resultados:

## Estimacion de Proporciones:

In [362]:
tBase = 118769

In [363]:
# Proporcion de Imagenes con Animales
pAnimal = 0.3188
print('Proporcion de Imagenes con Animales: ', round(pAnimal * 100,2), '%')

Proporcion de Imagenes con Animales:  31.88 %


In [364]:
# Proporcion de Imagenes sin Animales
pNoAnimal = 1 - pAnimal
print('Proporcion de Imagenes sin Animales: ', round(pNoAnimal * 100,2), '%')

Proporcion de Imagenes sin Animales:  68.12 %


### Proporciones a Validar

In [365]:
pValNoAnimal = 0.0406
print('Proporcion de Imagenes sin Animales a Validar: ', round(pValNoAnimal * 100,2), '%')

Proporcion de Imagenes sin Animales a Validar:  4.06 %


In [366]:
pValAnimal = 0.8094
print('Proporcion de Imagenes con Animales a Validar: ', round(pValAnimal * 100,2), '%')

Proporcion de Imagenes con Animales a Validar:  80.94 %


In [367]:
# Porcentaje de Imagenes a Validar:
pVal = 0.2857
print('Porcentaje de Imagenes a Validar: ', round(pVal * 100,2), '%')

Porcentaje de Imagenes a Validar:  28.57 %


In [370]:
cVal = tBase * pNoAnimal * pValNoAnimal + tBase * pAnimal * pValAnimal
print('Cantidad de Imagenes a Validar: ', math.ceil(cVal))

Cantidad de Imagenes a Validar:  33932


## Tiempo Total del Proceso:

In [281]:
# Ratio de 4.18 imagenes por segundo
tAutom = 1/4.18
print('Tiempo de Automatizacion: ', round(tAutom,2), 'segundos por imagen')

Tiempo de Automatizacion:  0.24 segundos por imagen


In [282]:
TAutom = (tBase * (tAutom))/(60*60)
print('Tiempo de Automatizacion: ', round(TAutom,2), 'horas')
print('Tiempo de Automatizacion: ', math.ceil(TAutom/8), 'jornadas')

Tiempo de Automatizacion:  7.89 horas
Tiempo de Automatizacion:  1 jornadas


Por lo que el tiempo total del proceso automatico es de **7.89 horas**  que son equivalentes a **1 jornada**.

### Tiempo de Validacion:

El porcantaje de imagenes a validar sobre el total de la base es de:

In [283]:
# Tiempo de Validacion:
tVal = pVal * 1.36
TVal = (tBase * tVal)/(60*60)
print('Tiempo de Validacion: ', round(TVal,2), 'horas')
print('Tiempo de Validacion: ', math.ceil(TVal/8), 'jornadas')

Tiempo de Validacion:  12.82 horas
Tiempo de Validacion:  2 jornadas


Por lo que el tiempo total del proceso de validacion es de **14.85 horas**  que son equivalentes a **2 jornadas**.

### Tiempo Total

El tiempo total del proceso sera de:

In [284]:
tTotal = tAutom + tVal
TTotal = TAutom + TVal
print('Tiempo del Proceso: ', round(TTotal,2), 'horas')
print('Tiempo del Proceso: ', round(TTotal/8,2), 'jornadas')

Tiempo del Proceso:  20.71 horas
Tiempo del Proceso:  2.59 jornadas


El proceso completo requiere **2.84 jornadas laborales**  (23 horas). Cabe destacar que de estas solo **15 horas requieren atencion humana**

### Cantidad de Imagenes por Hora:

Para calcular la cantidad de imágenes que se procesaran por hora se tiene en cuenta tanto el tiempo que tarda en procesarlas los modelos predictivos cómo el tiempo que se tarda en validar aquellas que necesiten un refuerzo en su clasificación. 

In [285]:
print('Tiempo de Automatizacion: ', round(tAutom,2), 'segundos')

cHora = 60*60/(tAutom + tVal)
print('Cantidad de Imagenes por Hora: ', math.ceil(cHora))

Tiempo de Automatizacion:  0.24 segundos
Cantidad de Imagenes por Hora:  5735


### Precision de Etiquetado:

Precisión:
- Presencias: 96.55%
- Especies: 91.43%
- Cantidades: 87.04%

In [286]:
aPresencias = 96.55
aEspecies = 91.43
aCantidades = 87.04

Seguridad de Prediccion:
- Presencias:
  - Animal: 92.74%
  - No Animal: 95.94%
- Especies: 55.34%
- Cantidades: 41.79%

In [287]:
sAnimal = 92.74/100
sNoAnimal = 95.94/100
sEspecies = 55.34/100
sCantidades = 41.79/100

Errores que No se Validan:
- Presencias:
  - Animal: 2.38%
  - No Animal: 1.45%
- Especies: 1.41%
- Cantidades: 5.22%

In [288]:
erAnimal = 2.38/100
erNoAnimal = 1.45/100
erEspecies = 1.41/100
erCantidades = 5.22/100

In [289]:
eNoAnimal = pNoAnimal * sNoAnimal * erNoAnimal
print('Error de Imagenes sin Animales: ', round(eNoAnimal * 100,2), '%')

Error de Imagenes sin Animales:  0.95 %


In [290]:
eAnimal = pAnimal * sEspecies * (erEspecies + sCantidades * erCantidades)
print('Error de Imagenes con Animales: ', round(eAnimal * 100,2), '%')

Error de Imagenes con Animales:  0.63 %


In [291]:
eAutomatico = eNoAnimal + eAnimal
print('Error Automatico: ', round(eAutomatico * 100,2), '%')

Error Automatico:  1.58 %


In [292]:
cErroresAutom = tBase * eAutomatico
print('Cantidad de Errores: ', math.ceil(cErroresAutom))

Cantidad de Errores:  1879


Tasa de Error en Validacion: Se asume que las tasas de error en el proceso de validación son 50% menores que en la situación actual. 
- Presencias:
  - FP: 0.3%
  - FN: 8.35%
- Especies: 1.7%
- Cantidades: 5%

In [293]:
eFP = 0.3/100
eFN = 8.35/100
eGuanacos = 1.7/100
eCantidades = 5/100

In [294]:
eValidacion = pAnimal * (eFN + eGuanacos + eCantidades) * pValAnimal + pNoAnimal * eCantidades * pValNoAnimal
print('Error de Validacion: ', round(eValidacion * 100,2), '%')

Error de Validacion:  4.02 %


In [295]:
cErroresVal = tBase * pVal * eValidacion
print('Cantidad de Errores: ', math.ceil(cErroresVal))

Cantidad de Errores:  1365


In [296]:
eErrores = eAutomatico + eValidacion
print('Error Total: ', round(eErrores * 100,2), '%')

Error Total:  5.6 %


In [297]:
cErrores = cErroresAutom + cErroresVal
print('Cantidad de Errores: ', math.ceil(cErrores))

Cantidad de Errores:  3243


### Costos del Proceso:

- Costo Falso Negativo: $300
- Costo Falso Positivo: $1000
- Costo Error en Especies: $1300
- Costo Error en Cantidad: $650

In [298]:
coErrorNoAnimal = eNoAnimal * 300
print('Costo de Error de Imagenes sin Animales: $', round(coErrorNoAnimal,2))

Costo de Error de Imagenes sin Animales: $ 2.84


In [299]:
coErrorAnimal = pAnimal * sEspecies * (erEspecies * 1300 + sCantidades * erCantidades * 650)
print('Costo de Error de Imagenes con Animales: $', round(coErrorAnimal,2))

Costo de Error de Imagenes con Animales: $ 5.74


In [300]:
coErrorAutomatico = coErrorNoAnimal + coErrorAnimal
print('Costo de Error Automatico: $', round(coErrorAutomatico,2))

Costo de Error Automatico: $ 8.58


In [301]:
coErrorVal = pAnimal * pValAnimal * erEspecies * 300 + (sCantidades * erCantidades * 650) + pNoAnimal * pValNoAnimal * eFP * 1000
print('Costo de Error de Validacion: $', round(coErrorVal,2))

Costo de Error de Validacion: $ 15.35


In [302]:
coOperacionalVal = pVal * tVal * 10730/(8*60*60)
print('Costo Operacional de Validacion: $', round(coOperacionalVal,2))

Costo Operacional de Validacion: $ 0.04


In [303]:
coServicio = 1100000
print('Costo de Servicio: $', round(coServicio,2))

Costo de Servicio: $ 1100000


In [304]:
coTotal = tBase * (coErrorAutomatico + coErrorVal + coOperacionalVal) + coServicio
print('Costo Total: $', round(coTotal,2))

Costo Total: $ 3947309.91


In [305]:
coImagen = coTotal/tBase
print('Costo por Imagen: $', round(coImagen,2))

Costo por Imagen: $ 33.24


## KPIs

### Tiempo Total del Proceso

In [306]:
print('Tiempo Total: ')
print('- Clasificacion Manual: 152.75 horas')
print('- Resultados Semi-Automatica:', round(TTotal,2), 'horas')
print('Variacion Porcentual: ', round((TTotal - 152.75)/152.75 * 100,2), '%')

Tiempo Total: 
- Clasificacion Manual: 152.75 horas
- Resultados Semi-Automatica: 20.71 horas
Variacion Porcentual:  -86.44 %


In [307]:
print('Jornadas Laborales:')
print('- Clasificacion Manual: 19.1 jornadas')
print('- Resultados Semi-Automatica:')
print('   - Jornadas Ocupadas: ', round(TVal/8,1), 'jornadas')
print('    Variacion Porcentual: ', round((TVal/8 - 19.1)/19.1 * 100,2), '%')
print('    - Jornadas Totales: ', round(TTotal/8,1), 'jornadas')
print('    Variacion Porcentual: ', round((TTotal/8 - 19.1)/19.1 * 100,2), '%')

Jornadas Laborales:
- Clasificacion Manual: 19.1 jornadas
- Resultados Semi-Automatica:
   - Jornadas Ocupadas:  1.6 jornadas
    Variacion Porcentual:  -91.61 %
    - Jornadas Totales:  2.6 jornadas
    Variacion Porcentual:  -86.45 %


### Cantidad de Imagenes Analizadas por Hora:

In [308]:
print('Cantidad de Imagenes por Hora: ')
print('- Clasificacion Manual: 777 imagenes/hr')
print('- Resultados Semi-Automatica: ', math.ceil(cHora), 'imagenes/hr')
print('Variacion Porcentual: ', round((cHora - 777)/777 * 100,2), '%')

Cantidad de Imagenes por Hora: 
- Clasificacion Manual: 777 imagenes/hr
- Resultados Semi-Automatica:  5735 imagenes/hr
Variacion Porcentual:  638.02 %


### Precision de Etiquetado:

In [309]:
print('Tasa de Error: ')
print('- Clasificacion Manual: 10%')
print('- Resultados Semi-Automatica: ', round(eErrores * 100,2), '%')
print('Variacion Porcentual: ', round((eErrores * 100 - 10)/10 * 100,2), '%')

Tasa de Error: 
- Clasificacion Manual: 10%
- Resultados Semi-Automatica:  5.6 %
Variacion Porcentual:  -43.97 %


In [310]:
print('Cantidad de Errores: ')
print('- Clasificacion Manual: 11877 errores')
print('- Resultados Semi-Automatica: ', math.ceil(cErrores), 'errores')
print('Variacion Porcentual: ', round((cErrores - 11877)/11877 * 100,2), '%')

Cantidad de Errores: 
- Clasificacion Manual: 11877 errores
- Resultados Semi-Automatica:  3243 errores
Variacion Porcentual:  -72.7 %


### Costo del Proceso:

In [311]:
print('Costo por Imagen:')
print('- Clasificacion Manual: $ 56.5')
print('- Resultados Semi-Automatica: $', round(coImagen,2))
print('Variacion Porcentual: ', round((coImagen - 56.5)/56.5* 100,2), '%')

Costo por Imagen:
- Clasificacion Manual: $ 56.5
- Resultados Semi-Automatica: $ 33.24
Variacion Porcentual:  -41.18 %


In [312]:
print('Costo del Proceso:')
print('- Clasificacion Manual: $ 6722325.4')
print('- Resultados Semi-Automatica: $', round(coTotal,2))
print('Variacion Porcentual: ', round((coTotal - 6722325.4)/6722325.4 * 100,2), '%')

Costo del Proceso:
- Clasificacion Manual: $ 6722325.4
- Resultados Semi-Automatica: $ 3947309.91
Variacion Porcentual:  -41.28 %


### Ratio de Imagenes Utiles:

In [313]:
print('Ratio de Imagenes Utiles: ')
print('- Clasificacion Manual: 0.79%')
print('- Resultados Semi-Automatica: 100%')
print('Variacion Porcentual: ', round((100 - 0.79)/0.79 * 100,2), '%')

Ratio de Imagenes Utiles: 
- Clasificacion Manual: 0.79%
- Resultados Semi-Automatica: 100%
Variacion Porcentual:  12558.23 %


In [314]:
ResultadosSemiAutomatica = [round(tTotal,2), round(TVal/8,1), round(TTotal/8,1), math.ceil(cHora), round(eErrores * 100,2), math.ceil(cErrores), round(coImagen,2), 100]

# Resultados Piloto 2023


In [371]:
df = pd.read_csv('Muestreo_CT_PatAzul2023.csv')

/var/folders/6c/17qbm4nd5318hd4bcrx68wjh0000gn/T/ipykernel_66877/1758670802.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Muestreo_CT_PatAzul2023.csv')


In [316]:
df.columns

Index(['Unnamed: 0', 'Ruta', 'Sitio', 'Año', 'Camara', 'Extra', 'Archivo',
       'Fecha', 'Hora', 'Animal_proba', 'Animal', 'Guanaco_proba', 'Guanaco',
       'Especie', 'Cantidad_proba', 'Cantidad', 'Validar', 'Validado'],
      dtype='object')

In [317]:
df.shape

(118765, 18)

## Estimacion de Proporciones:

In [318]:
# Proporcion de Imagenes con Animales
pAnimal = sum(df['Animal']==1)/len(df)
print('Proporcion de Imagenes con Animales: ', round(pAnimal * 100,2), '%')

Proporcion de Imagenes con Animales:  29.2 %


In [319]:
# Proporcion de Imagenes sin Animales
pNoAnimal = sum(df['Animal']==0)/len(df)
print('Proporcion de Imagenes sin Animales: ', round(pNoAnimal * 100,2), '%')

Proporcion de Imagenes sin Animales:  70.8 %


### Proporciones a Validar

In [320]:
pValNoAnimal = sum((df['Validar']==1) & (df['Animal']==0))/sum(df['Animal']==0)
print('Proporcion de Imagenes sin Animales a Validar: ', round(pValNoAnimal * 100,2), '%')

Proporcion de Imagenes sin Animales a Validar:  13.93 %


In [321]:
pValAnimal = sum((df['Validar']==1) & (df['Animal']==1))/sum(df['Animal']==1)
print('Proporcion de Imagenes con Animales a Validar: ', round(pValAnimal * 100,2), '%')

Proporcion de Imagenes con Animales a Validar:  79.58 %


In [322]:
# Porcentaje de Imagenes a Validar:
pVal = sum(df['Validar']==1)/len(df)
print('Porcentaje de Imagenes a Validar: ', round(pVal * 100,2), '%')

Porcentaje de Imagenes a Validar:  33.1 %


## Tiempo Total del Proceso:

### Tiempo de Ejecucion Automatica:

La eficiencia en los tiempos se vio perjudica por el uso de AWS para almacenar las imagenes. Al tener mayor latencia el proceso de consultar las imagenes demora más. Ademas, para consultar las imagenes se debieron tensorificar y extraer los metadatos antes de aplicar los modelos.
Se utilizaron multiprocesos concurrentes para hacer el proceso mas rapido. Y tecnicas de autoguardado para que en caso de que se rompa el proceso no se tenga que iniciar desde cero.

In [323]:
# Ratio de 4.18 imagenes por segundo
tAutom = 1/4.18
print('Tiempo de Automatizacion: ', round(tAutom,2), 'segundos por imagen')

Tiempo de Automatizacion:  0.24 segundos por imagen


In [324]:
TAutom = (len(df) * (tAutom))/(60*60)
print('Tiempo de Automatizacion: ', round(TAutom,2), 'horas')
print('Tiempo de Automatizacion: ', math.ceil(TAutom/8), 'jornadas')

Tiempo de Automatizacion:  7.89 horas
Tiempo de Automatizacion:  1 jornadas


Por lo que el tiempo total del proceso automatico es de **7.89 horas**  que son equivalentes a **1 jornada**.

### Tiempo de Validacion:

El porcantaje de imagenes a validar sobre el total de la base es de:

In [325]:
# Tiempo de Validacion:
tVal = pVal * 1.36
TVal = (len(df) * tVal)/(60*60)
print('Tiempo de Validacion: ', round(TVal,2), 'horas')
print('Tiempo de Validacion: ', math.ceil(TVal/8), 'jornadas')

Tiempo de Validacion:  14.85 horas
Tiempo de Validacion:  2 jornadas


Por lo que el tiempo total del proceso de validacion es de **14.85 horas**  que son equivalentes a **2 jornadas**.

### Tiempo Total

El tiempo total del proceso sera de:

In [326]:
tTotal = tAutom + tVal
TTotal = TAutom + TVal
print('Tiempo del Proceso: ', round(TTotal,2), 'horas')
print('Tiempo del Proceso: ', round(TTotal/8,2), 'jornadas')

Tiempo del Proceso:  22.74 horas
Tiempo del Proceso:  2.84 jornadas


El proceso completo requiere **2.84 jornadas laborales**  (23 horas). Cabe destacar que de estas solo **15 horas requieren atencion humana**

### Cantidad de Imagenes por Hora:

Para calcular la cantidad de imágenes que se procesaran por hora se tiene en cuenta tanto el tiempo que tarda en procesarlas los modelos predictivos cómo el tiempo que se tarda en validar aquellas que necesiten un refuerzo en su clasificación. 

In [327]:
print('Tiempo de Automatizacion: ', round(tAutom,2), 'segundos')

cHora = 60*60/(tAutom + tVal)
print('Cantidad de Imagenes por Hora: ', math.ceil(cHora))

Tiempo de Automatizacion:  0.24 segundos
Cantidad de Imagenes por Hora:  5222


### Precision de Etiquetado:

Precisión:
- Presencias: 96.55%
- Especies: 91.43%
- Cantidades: 87.04%

In [328]:
aPresencias = 96.55
aEspecies = 91.43
aCantidades = 87.04

Seguridad de Prediccion:
- Presencias:
  - Animal: 92.74%
  - No Animal: 95.94%
- Especies: 55.34%
- Cantidades: 41.79%

In [329]:
sAnimal = 92.74/100
sNoAnimal = 95.94/100
sEspecies = 55.34/100
sCantidades = 41.79/100

Errores que No se Validan:
- Presencias:
  - Animal: 2.38%
  - No Animal: 1.45%
- Especies: 1.41%
- Cantidades: 5.22%

In [330]:
erAnimal = 2.38/100
erNoAnimal = 1.45/100
erEspecies = 1.41/100
erCantidades = 5.22/100

In [331]:
eNoAnimal = pNoAnimal * sNoAnimal * erNoAnimal
print('Error de Imagenes sin Animales: ', round(eNoAnimal * 100,2), '%')

Error de Imagenes sin Animales:  0.98 %


In [332]:
eAnimal = pAnimal * sEspecies * (erEspecies + sCantidades * erCantidades)
print('Error de Imagenes con Animales: ', round(eAnimal * 100,2), '%')

Error de Imagenes con Animales:  0.58 %


In [333]:
eAutomatico = eNoAnimal + eAnimal
print('Error Automatico: ', round(eAutomatico * 100,2), '%')

Error Automatico:  1.57 %


In [334]:
cErroresAutom = len(df) * eAutomatico
print('Cantidad de Errores: ', math.ceil(cErroresAutom))

Cantidad de Errores:  1860


Tasa de Error en Validacion: Se asume que las tasas de error en el proceso de validación son 50% menores que en la situación actual. 
- Presencias:
  - FP: 0.3%
  - FN: 8.35%
- Especies: 1.7%
- Cantidades: 5%

In [335]:
eFP = 0.3/100
eFN = 8.35/100
eGuanacos = 1.7/100
eCantidades = 5/100

In [336]:
eValidacion = pAnimal * (eFN + eGuanacos + eCantidades) * pValAnimal + pNoAnimal * eCantidades * pValNoAnimal
print('Error de Validacion: ', round(eValidacion * 100,2), '%')

Error de Validacion:  3.99 %


In [337]:
cErroresVal = len(df) * pVal * eValidacion 
print('Cantidad de Errores: ', math.ceil(cErroresVal))

Cantidad de Errores:  1570


In [338]:
eErrores = eAutomatico + eValidacion
print('Error Total: ', round(eErrores * 100,2), '%')

Error Total:  5.56 %


In [339]:
cErrores = cErroresAutom + cErroresVal
print('Cantidad de Errores: ', math.ceil(cErrores))

Cantidad de Errores:  3429


### Costos del Proceso:

- Costo Falso Negativo: $300
- Costo Falso Positivo: $1000
- Costo Error en Especies: $1300
- Costo Error en Cantidad: $650

In [340]:
coErrorNoAnimal = eNoAnimal * 300
print('Costo de Error de Imagenes sin Animales: $', round(coErrorNoAnimal,2))

Costo de Error de Imagenes sin Animales: $ 2.95


In [341]:
coErrorAnimal = pAnimal * sEspecies * (erEspecies * 1300 + sCantidades * erCantidades * 650)
print('Costo de Error de Imagenes con Animales: $', round(coErrorAnimal,2))

Costo de Error de Imagenes con Animales: $ 5.25


In [342]:
coErrorAutomatico = coErrorNoAnimal + coErrorAnimal
print('Costo de Error Automatico: $', round(coErrorAutomatico,2))

Costo de Error Automatico: $ 8.21


In [343]:
coErrorVal = pAnimal * pValAnimal * erEspecies * 300 + (sCantidades * erCantidades * 650) + pNoAnimal * pValNoAnimal * eFP * 1000
print('Costo de Error de Validacion: $', round(coErrorVal,2))

Costo de Error de Validacion: $ 15.46


In [344]:
coOperacionalVal = pVal * tVal * 10730/(8*60*60)
print('Costo Operacional de Validacion: $', round(coOperacionalVal,2))

Costo Operacional de Validacion: $ 0.06


In [345]:
coServicio = 1100000
print('Costo de Servicio: $', round(coServicio,2))

Costo de Servicio: $ 1100000


In [346]:
coTotal = len(df) * (coErrorAutomatico + coErrorVal + coOperacionalVal) + coServicio
print('Costo Total: $', round(coTotal,2))

Costo Total: $ 3917405.1


In [347]:
coImagen = coTotal/len(df)
print('Costo por Imagen: $', round(coImagen,2))

Costo por Imagen: $ 32.98


## KPIs

### Tiempo Total del Proceso

In [348]:
print('Tiempo Total: ')
print('- Clasificacion Manual: 152.75 horas')
print('- Solucion Semi-Automatica:', round(TTotal,2), 'horas')
print('Variacion Porcentual: ', round((TTotal - 152.75)/152.75 * 100,2), '%')

Tiempo Total: 
- Clasificacion Manual: 152.75 horas
- Solucion Semi-Automatica: 22.74 horas
Variacion Porcentual:  -85.11 %


In [349]:
print('Jornadas Laborales:')
print('- Clasificacion Manual: 19.1 jornadas')
print('- Solucion Semi-Automatica:')
print('   - Jornadas Ocupadas: ', round(TVal/8,1), 'jornadas')
print('    Variacion Porcentual: ', round((TVal/8 - 19.1)/19.1 * 100,2), '%')
print('    - Jornadas Totales: ', round(TTotal/8,1), 'jornadas')
print('    Variacion Porcentual: ', round((TTotal/8 - 19.1)/19.1 * 100,2), '%')

Jornadas Laborales:
- Clasificacion Manual: 19.1 jornadas
- Solucion Semi-Automatica:
   - Jornadas Ocupadas:  1.9 jornadas
    Variacion Porcentual:  -90.28 %
    - Jornadas Totales:  2.8 jornadas
    Variacion Porcentual:  -85.11 %


### Cantidad de Imagenes Analizadas por Hora:

In [350]:
print('Cantidad de Imagenes por Hora: ')
print('- Clasificacion Manual: 777 imagenes/hr')
print('- Solucion Semi-Automatica: ', math.ceil(cHora), 'imagenes/hr')
print('Variacion Porcentual: ', round((cHora - 777)/777 * 100,2), '%')

Cantidad de Imagenes por Hora: 
- Clasificacion Manual: 777 imagenes/hr
- Solucion Semi-Automatica:  5222 imagenes/hr
Variacion Porcentual:  572.03 %


### Precision de Etiquetado:

In [351]:
print('Tasa de Error: ')
print('- Clasificacion Manual: 10%')
print('- Solucion Semi-Automatica: ', round(eErrores * 100,2), '%')
print('Variacion Porcentual: ', round((eErrores * 100 - 10)/10 * 100,2), '%')

Tasa de Error: 
- Clasificacion Manual: 10%
- Solucion Semi-Automatica:  5.56 %
Variacion Porcentual:  -44.44 %


In [352]:
print('Cantidad de Errores: ')
print('- Clasificacion Manual: 11877 errores')
print('- Solucion Semi-Automatica: ', math.ceil(cErrores), 'errores')
print('Variacion Porcentual: ', round((cErrores - 11877)/11877 * 100,2), '%')

Cantidad de Errores: 
- Clasificacion Manual: 11877 errores
- Solucion Semi-Automatica:  3429 errores
Variacion Porcentual:  -71.14 %


### Costo del Proceso:

In [353]:
print('Costo por Imagen:')
print('- Clasificacion Manual: $ 56.5')
print('- Solucion Semi-Automatica: $', round(coImagen,2))
print('Variacion Porcentual: ', round((coImagen - 56.5)/56.5* 100,2), '%')

Costo por Imagen:
- Clasificacion Manual: $ 56.5
- Solucion Semi-Automatica: $ 32.98
Variacion Porcentual:  -41.62 %


In [354]:
print('Costo del Proceso:')
print('- Clasificacion Manual: $ 6722325.4')
print('- Solucion Semi-Automatica: $', round(coTotal,2))
print('Variacion Porcentual: ', round((coTotal - 6722325.4)/6722325.4 * 100,2), '%')

Costo del Proceso:
- Clasificacion Manual: $ 6722325.4
- Solucion Semi-Automatica: $ 3917405.1
Variacion Porcentual:  -41.73 %


### Ratio de Imagenes Utiles:

In [355]:
print('Ratio de Imagenes Utiles: ')
print('- Clasificacion Manual: 0.79%')
print('- Solucion Semi-Automatica: 100%')
print('Variacion Porcentual: ', round((100 - 0.79)/0.79 * 100,2), '%')

Ratio de Imagenes Utiles: 
- Clasificacion Manual: 0.79%
- Solucion Semi-Automatica: 100%
Variacion Porcentual:  12558.23 %


In [356]:
solucionSemiAutomatica=[round(tTotal,2), round(TVal/8,1), round(TTotal/8,1), math.ceil(cHora), round(eErrores * 100,2), math.ceil(cErrores), round(coImagen,2), 100 ]

# Resultados:

In [357]:
df = pd.read_csv('resultados.csv')

In [358]:
df['Resultados Semi-Automatica'] = ResultadosSemiAutomatica

In [359]:
df["Solucion Semi-Automatica"] = solucionSemiAutomatica

In [360]:
df["Variacion Porcentual (%)"] = round((df["Solucion Semi-Automatica"] - df["Clasificacion Manual"]) / df["Clasificacion Manual"] * 100, 2)
df

,KPI,Clasificacion Manual,Estimacion Automatica,Estimacion Semi-Automatica,Resultados Semi-Automatica,Solucion Semi-Automatica,Variacion Porcentual (%)
0,Segundos por Imagen,4.63,0.56,1.84,0.63,0.69,-85.10
1,Jornadas Ocupadas,19.10,0.00,1.60,1.60,1.90,-90.05
2,Jornadas Totales,19.10,2.30,7.60,2.60,2.80,-85.34
3,Imagenes por Hora,777.00,6480.00,1962.00,5735.00,5222.00,572.07
4,Tasa de Error,10.00,16.30,6.13,5.60,5.56,-44.40
5,Cantidad de Errores,11877.00,19357.00,7281.00,3243.00,3429.00,-71.13
6,Costo por Imagen,56.50,115.59,22.69,33.24,32.98,-41.63
7,Ratio de Imagenes Utiles,0.79,100.00,100.00,100.00,100.00,12558.23
